In [1]:
import numpy as np
from braindecode.datasets.moabb import MOABBDataset
from braindecode.datautil.preprocess import exponential_moving_standardize
from braindecode.datautil.preprocess import MNEPreproc
from braindecode.datautil.preprocess import NumpyPreproc
from braindecode.datautil.preprocess import preprocess
from braindecode.datautil.windowers import create_windows_from_events

import os
from glob import glob

import sys

In [2]:
def print_off():
    sys.stdout = open(os.devnull, 'w')
def print_on():
    sys.stdout = sys.__stdout__

In [3]:
def bcic4_2a(subject, low_hz=None, high_hz=None):
    X = []
    y = []
    if isinstance(subject, int):
        subject = [subject]
    for subject_id in subject:
        # Load data
        print_off()
        dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=[subject_id])
        # Preprocess
        factor_new = 1e-3
        init_block_size = 1000
        preprocessors = [
            # keep only EEG sensors
            MNEPreproc(fn='pick_types', eeg=True, meg=True, stim=False),
            # convert from volt to microvolt
            NumpyPreproc(fn=lambda x: x * 1e+06),
            # bandpass filter
            MNEPreproc(fn='filter', l_freq=low_hz, h_freq=high_hz),
            # exponential moving standardization
            NumpyPreproc(fn=exponential_moving_standardize, factor_new=factor_new,
                         init_block_size=init_block_size)
        ]
        preprocess(dataset, preprocessors)
        # Divide data by trial
        # - Check sampling frequency
        sfreq = dataset.datasets[0].raw.info['sfreq']
        assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
        trial_start_offset_seconds = -0.5
        trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)
        windows_dataset = create_windows_from_events(
            dataset,
            trial_start_offset_samples=trial_start_offset_samples,
            trial_stop_offset_samples=0,
            preload=True
            # verbose=True
        )
        # If you need split data, try this.
        splitted = windows_dataset.split('session')
        train_set = splitted['session_T']
        valid_set = splitted['session_E']
        If you need concat data, try this.
        dataset = BaseConcatDataset([train_set, valid_set])
        # merge subject
        # data_bundle.append(windows_dataset)
        
        # Merge subject
        
        #for trial in windows_dataset:
            X.append(trial[0])
            y.append(trial[1])
    print_on()
    return np.array(X), np.array(y)

In [4]:
x, y = bcic4_2a([1,2,3], 1, 38)

In [5]:
x.shape

(1728, 22, 1125)

In [22]:
y.shape

(576,)

In [24]:
np.unique(y, return_counts=True)

(array([0, 1, 2, 3]), array([144, 144, 144, 144], dtype=int64))

In [9]:
val_subject = [1]
train_subject = list(set(range(1, 10)) - set(val_subject))

In [10]:
train_subject

[2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
x, y = bcic4_2a(train_subject)

In [2]:
print("데이터 확인용입니다", x.shape)

NameError: name 'x' is not defined

In [13]:
y.shape

(4608,)

In [18]:
"Hello"

'Hello'

In [3]:
print("Hello", 3)

Hello 3


In [9]:
import numpy as np

X = np.random.rand(576, 25, 1125)

In [7]:
X = np.reshape(X, [576,25,1125,1])

In [8]:
X.shape

(576, 25, 1125, 1)

In [10]:
X = np.expand_dims(X, -1)

In [11]:
X.shape

(576, 25, 1125, 1)